In [1]:
import os
import sys

project_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..'))
sys.path.insert(0, project_path)
print(f"Project path: {project_path}")

Project path: c:\Users\dalej\Documents\_Coding\DragonRegen


In [ ]:
from src.MetaFort.AILoggingTopics import AILoggingTopics
from kafka import Kafkaconsumer_batch
import json

kc_completed_task = Kafkaconsumer_batch(
    AILoggingTopics.AI_TASK_COMPLETED_TOPIC,
    bootstrap_servers=['localhost:9092'],
    auto_offset_reset='latest',
    enable_auto_commit=True,
    group_id='dwh-stream-job',
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)


In [32]:
completed = kc_completed_task.poll(timeout_ms=1000)
print("Messages received:")
for topic_partition, messages in completed.items():
    for message in messages:
        print(f"Topic: {topic_partition.topic}, Partition: {topic_partition.partition}, Offset: {message.offset}, Key: {message.key}, Value: {message.value}")
        

Messages received:
Topic: ai_tasks_completed, Partition: 0, Offset: 3, Key: None, Value: {'task_id': 300304694183836177550454581032852541384, 'task_name': 'DataColumnType', 'group_task_id': None, 'insert_dt': '2025-05-14T14:56:51.140983', 'output_artifacts': '{"reason": "The \'membership\' column represents the kind of membership a gym member has, which would be a limited set of predefined options (like \'Basic\', \'Premium\', \'Gold\', etc.) rather than free-form text. This is a classic example of a categorical variable with a finite number of possible values.", "col_type": "Categorical"}'}
Topic: ai_tasks_completed, Partition: 0, Offset: 4, Key: None, Value: {'task_id': 136058658848764322081540275259036020310, 'task_name': 'DataColumnType', 'group_task_id': None, 'insert_dt': '2025-05-14T15:06:28.809828', 'output_artifacts': '{"reason": "The \'membership\' column represents the kind of membership a gym member has, which would be a limited set of predefined options (like \'Basic\', \'

In [ ]:
from kafka import KafkaConsumer
from src.MetaFort.AILoggingTopics import AILoggingTopics
    
# Initialize consumer
consumer = KafkaConsumer(
    AILoggingTopics.AI_TASK_COMPLETED_TOPIC,
    bootstrap_servers=['localhost:9092'],
    auto_offset_reset='earliest',
    enable_auto_commit=False,
    group_id='record_puller'
)

records = {}
# Pull records
for message in consumer:
    # Assuming message.key as the record identifier and message.value as the record data
    key = message.key.decode('utf-8') if message.key else f"record_{len(records)}"
    value = message.value.decode('utf-8')
    records[key] = value

In [4]:
print(len(records))
print(records)

42
{'record_0': '{"task_id": 149559886724366975439206724654739506232, "task_name": "TaskSayHello", "group_task_id": null, "insert_dt": "2025-05-14T13:32:27.480557", "output_artifacts": "null"}', 'record_1': '{"task_id": 35730254192003998894948829634200368416, "task_name": "TaskSayHello", "group_task_id": null, "insert_dt": "2025-05-14T13:38:51.553211", "output_artifacts": "null"}', 'record_2': '{"task_id": 35730254192003998894948829634200368416, "task_name": "TaskSayHello", "group_task_id": null, "insert_dt": "2025-05-14T13:39:58.054279", "output_artifacts": "null"}', 'record_3': '{"task_id": 300304694183836177550454581032852541384, "task_name": "DataColumnType", "group_task_id": null, "insert_dt": "2025-05-14T14:56:51.140983", "output_artifacts": "{\\"reason\\": \\"The \'membership\' column represents the kind of membership a gym member has, which would be a limited set of predefined options (like \'Basic\', \'Premium\', \'Gold\', etc.) rather than free-form text. This is a classic ex

In [ ]:
from kafka import KafkaConsumer, TopicPartition
from src.MetaFort.AILoggingTopics import AILoggingTopics
    
topic_name = AILoggingTopics.AI_TASK_COMPLETED_TOPIC

# Initialize consumer
consumer_batch = KafkaConsumer(
    topic_name,
    bootstrap_servers=['localhost:9092'],
    auto_offset_reset='earliest',
    enable_auto_commit=False,
    group_id='batch_puller'
)

records = {}
raw_records = []
record_count = 0
batch_size = 22
max_records = 200


In [62]:
batch = consumer_batch.poll(timeout_ms=5000, max_records=batch_size)


In [75]:
print(batch)
print(type(batch))
print(batch.keys())
print(batch.values())
print(type(batch.values()))
vals = list(batch.values())
print(len(vals))
print(vals[0])
print(len(vals[0]))
print(type(vals[0][0]))


{TopicPartition(topic='ai_tasks_completed', partition=0): [ConsumerRecord(topic='ai_tasks_completed', partition=0, offset=0, timestamp=1747243947580, timestamp_type=0, key=None, value=b'{"task_id": 149559886724366975439206724654739506232, "task_name": "TaskSayHello", "group_task_id": null, "insert_dt": "2025-05-14T13:32:27.480557", "output_artifacts": "null"}', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=175, serialized_header_size=-1), ConsumerRecord(topic='ai_tasks_completed', partition=0, offset=1, timestamp=1747244331649, timestamp_type=0, key=None, value=b'{"task_id": 35730254192003998894948829634200368416, "task_name": "TaskSayHello", "group_task_id": null, "insert_dt": "2025-05-14T13:38:51.553211", "output_artifacts": "null"}', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=174, serialized_header_size=-1), ConsumerRecord(topic='ai_tasks_completed', partition=0, offset=2, timestamp=1747244398054, timestamp_type=0, key=None, v

In [80]:
reg_dict = {'a': 1, 'b': 2, 'c': 3}
print(reg_dict)
print(type(reg_dict))
print(reg_dict.keys())
print(reg_dict.values())

{'a': 1, 'b': 2, 'c': 3}
<class 'dict'>
dict_keys(['a', 'b', 'c'])
dict_values([1, 2, 3])


In [100]:
raw_records = []

 # Get partitions for the topic
partitions = consumer_batch.partitions_for_topic(topic_name)
print(f"Partitions for topic {topic_name}: {partitions}")
if not partitions:
    print(f"No partitions found for topic {topic_name}")

# Assign consumer to topic partitions
topic_partitions = [TopicPartition(topic_name, p) for p in partitions]
print(f"Partitions to assign topic {topic_name}: {topic_partitions}")
# consumer_batch.assign(topic_partitions)

# Pull records in batches
while True:
    # Fetch a batch of messages
    batch = consumer_batch.poll(timeout_ms=5000, max_records=batch_size)
    
    print(f"==> Batch current record count: {record_count}")
    if not batch:
        break  # No more records available
    # Process the batch
    for partition, messages in batch.items():
        # raw_records = {**raw_records, **messages}
        raw_records.append(messages)
        for message in messages:
            key = message.key.decode('utf-8') if message.key else f"record_{record_count}"
            records[key] = message.value  # Already deserialized by consumer
            
            record_count += 1

Partitions for topic ai_tasks_completed: {0}
Partitions to assign topic ai_tasks_completed: [TopicPartition(topic='ai_tasks_completed', partition=0)]
==> Batch current record count: 0
==> Batch current record count: 22
==> Batch current record count: 44
==> Batch current record count: 66
==> Batch current record count: 80


In [101]:
print(len(records))
print(len(raw_records))

80
4


In [104]:
print(raw_records[0])
print(len(raw_records[0]))

[ConsumerRecord(topic='ai_tasks_completed', partition=0, offset=0, timestamp=1747243947580, timestamp_type=0, key=None, value=b'{"task_id": 149559886724366975439206724654739506232, "task_name": "TaskSayHello", "group_task_id": null, "insert_dt": "2025-05-14T13:32:27.480557", "output_artifacts": "null"}', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=175, serialized_header_size=-1), ConsumerRecord(topic='ai_tasks_completed', partition=0, offset=1, timestamp=1747244331649, timestamp_type=0, key=None, value=b'{"task_id": 35730254192003998894948829634200368416, "task_name": "TaskSayHello", "group_task_id": null, "insert_dt": "2025-05-14T13:38:51.553211", "output_artifacts": "null"}', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=174, serialized_header_size=-1), ConsumerRecord(topic='ai_tasks_completed', partition=0, offset=2, timestamp=1747244398054, timestamp_type=0, key=None, value=b'{"task_id": 35730254192003998894948829634200368416,

In [110]:
from datetime import datetime

for record in raw_records[3]:
  # Timestamp info
    timestamp_value = record.timestamp
    # Convert timestamp to datetime if needed
    timestamp_datetime = datetime.fromtimestamp(timestamp_value / 1000.0)
    print(f"Timestamp: {timestamp_datetime}, Timestamp Type: {record.timestamp_type}")
    # All record attributes above as one line
    print(f"Key: {record.key}, Offset: {record.offset}, Partition: {record.partition}, Topic: {record.topic}, Timestamp: {record.timestamp}, Timestamp Type: {record.timestamp_type}, Headers: {record.headers}, Value: {record.value}")

Timestamp: 2025-05-15 11:23:46.776000, Timestamp Type: 0
Key: None, Offset: 66, Partition: 0, Topic: ai_tasks_completed, Timestamp: 1747322626776, Timestamp Type: 0, Headers: [], Value: b'{"task_id": "a2c0a654-9596-4089-8f32-9162754c0ff0", "task_name": "DataColumnType", "group_task_id": "a2c0a654-9596-4089-8f32-9162754c0ff0", "insert_dt": "2025-05-15T11:23:46.775842", "output_artifacts": "{\\"reason\\": \\"The column stores a password hash, which is a secure representation of a user\'s password. Password hashes are typically stored as text strings that represent the cryptographic hash of the original password, making this a text field rather than a basic string with predefined format or a categorical value.\\", \\"col_type\\": \\"Text\\"}"}'
Timestamp: 2025-05-15 11:23:50.262000, Timestamp Type: 0
Key: None, Offset: 67, Partition: 0, Topic: ai_tasks_completed, Timestamp: 1747322630262, Timestamp Type: 0, Headers: [], Value: b'{"task_id": "2f3070e8-365f-4e79-999b-56be8827d8b2", "task_na

In [89]:
for key, value in records.items():
    print(f"Key: {key}, Value: {value.decode('utf-8') if isinstance(value, bytes) else value}")

Key: record_0, Value: {"task_id": 149559886724366975439206724654739506232, "task_name": "TaskSayHello", "group_task_id": null, "insert_dt": "2025-05-14T13:32:27.480557", "output_artifacts": "null"}
Key: record_1, Value: {"task_id": 35730254192003998894948829634200368416, "task_name": "TaskSayHello", "group_task_id": null, "insert_dt": "2025-05-14T13:38:51.553211", "output_artifacts": "null"}
Key: record_2, Value: {"task_id": 35730254192003998894948829634200368416, "task_name": "TaskSayHello", "group_task_id": null, "insert_dt": "2025-05-14T13:39:58.054279", "output_artifacts": "null"}
Key: record_3, Value: {"task_id": 300304694183836177550454581032852541384, "task_name": "DataColumnType", "group_task_id": null, "insert_dt": "2025-05-14T14:56:51.140983", "output_artifacts": "{\"reason\": \"The 'membership' column represents the kind of membership a gym member has, which would be a limited set of predefined options (like 'Basic', 'Premium', 'Gold', etc.) rather than free-form text. This 

In [34]:
# grab and make a list of all task_ids in records
task_ids = []
for task_json in records.values():
    task_json = json.loads(task_json.decode('utf-8'))
    task_id = task_json.get("task_id")
    task_ids.append(task_id)

In [35]:
print(task_ids)

[149559886724366975439206724654739506232, 35730254192003998894948829634200368416, 35730254192003998894948829634200368416, 300304694183836177550454581032852541384, 136058658848764322081540275259036020310, 104404014328354376757570259899498965908, 171699494359573712349098260085892114599, 291151643959420244732224483261331456813, 273117666406304310064435208146805318039, 34678693518986912951147276757046588208, 27544064338879441355909673248775178818, 35215827420894540719697955912594882671, 88427332889619924250208294660712435609, 228049317769154496570841790856413925401, 268207491734656532566507697014839312595, 82624278272589372523577399107114629736, 225517969496535558531431906343604493605, 323978463719078589832380212089555161626, 56470949197495537781695582159172636165, 309647346051781880921130622524073132078, 99494338120804711184271281679059535640, 247454130855242133410771342404125124074, 214483868217048755910560455845499701527, 186467540746977208656695237955578849988, 173667950627562730152076

In [39]:
id = 127669638701243457160312659282949941032
print(id in task_ids)
print(type(int(task_ids[0])))

False
<class 'int'>


In [37]:
ls = [1, 2, 3]
print(1 in ls)

True


In [53]:
child_task = ['a64ac0eb-d4c6-48a8-bbae-c1646539d2b9', '09e3c97b-1691-4096-8958-b4788387ee77', '9d5f73dc-e77b-4d9c-b4eb-13445f619c05', '2dfb6d02-0505-4237-a549-ab93546d989b', 'a8316924-794a-4b13-ab52-40aaec0ef925', '2295f9a6-53c1-4ebe-8859-b5d1775f6043', 'c58ee3e0-1b28-4365-9d0f-759c3b5ee738', '0ef98c8b-bc38-4731-af78-07796b2748b2', '63cec924-81f5-41f5-88be-1efc4b065041', 'ef6dc840-524e-4ea9-a227-09b10c090858', '7d951b63-dcc4-4886-9dff-9ea9831259fd', 'b62b120c-70ff-44bd-94f9-edb5e9152d32', 'f6a47f24-8158-4abc-bb69-3b5002b67b54', '24ecb93e-d064-4d46-abc4-1183a34d71c6', '6569252b-91d4-482a-a1de-ffcca20c4c49', '07a3ac9b-4dd1-4839-aa32-a24c9c104de3', '00703936-9873-498a-a6ab-cb42c09eac17', 'abbdca16-9079-4f36-8bab-eb59c63c3b20']
print('a64ac0eb-d4c6-48a8-bbae-c1646539d2b9' in child_task)

True


In [52]:
import json
# records
# child_task = [127669638701243457160312659282949941032, 37212638275987096423178173063332524120, 103182741281093254283710109008020428271, 76656977058476884230645155902388369626, 154726568387966792101450583700057459721, 307861089300525768249382168730600089870, 98240606463124509563787419306866987015, 136961009337957071396966281159969723686, 24680832926166466352636476901379417272, 118055550242447294570885734792333616895, 236245974605180568798023556630405234495, 20946505120684932936696150165153385857, 115754348903651463026244644083289116692, 45199626146624004423147197891567181723, 68844667136253279226332309499214836862, 145119008091859698494380896762582180692, 56669048749548959445558836830562033407, 315180359467859282242650439169195558525]
child_task = ['a64ac0eb-d4c6-48a8-bbae-c1646539d2b9', '09e3c97b-1691-4096-8958-b4788387ee77', '9d5f73dc-e77b-4d9c-b4eb-13445f619c05', '2dfb6d02-0505-4237-a549-ab93546d989b', 'a8316924-794a-4b13-ab52-40aaec0ef925', '2295f9a6-53c1-4ebe-8859-b5d1775f6043', 'c58ee3e0-1b28-4365-9d0f-759c3b5ee738', '0ef98c8b-bc38-4731-af78-07796b2748b2', '63cec924-81f5-41f5-88be-1efc4b065041', 'ef6dc840-524e-4ea9-a227-09b10c090858', '7d951b63-dcc4-4886-9dff-9ea9831259fd', 'b62b120c-70ff-44bd-94f9-edb5e9152d32', 'f6a47f24-8158-4abc-bb69-3b5002b67b54', '24ecb93e-d064-4d46-abc4-1183a34d71c6', '6569252b-91d4-482a-a1de-ffcca20c4c49', '07a3ac9b-4dd1-4839-aa32-a24c9c104de3', '00703936-9873-498a-a6ab-cb42c09eac17', 'abbdca16-9079-4f36-8bab-eb59c63c3b20']

child_task_output_artifacts = {}
# while child_task:

for _, message in records.items():
    # print(f"==> Processing message: {type(message.decode('utf-8'))} and  {message} ")
    if not isinstance(message, dict):
        task_json = json.loads(message.decode('utf-8'))
    else:
        task_json = message
    task_id = task_json.get("task_id")
    if task_id == "5ee44e70-93b2-4f84-b09e-55c7afeecfb8":
        print('Woot!')
    # print(f"==> Found task_id: {task_id} and {type(task_id)}")
    if task_id in child_task:
        print(f"==> Remove task_id: {task_id}")
        child_task.remove(task_id)
        child_task_output_artifacts[task_id] = task_json.get("output_artifacts", None)

Woot!


In [59]:
for _, message in records.items():
    # print(f"==> Processing message: {type(message.decode('utf-8'))} and  {message} ")
    if not isinstance(message, dict):
        task_json = json.loads(message.decode('utf-8'))
    else:
        task_json = message
    task_id = task_json.get("task_id")
    if isinstance(task_id, str):
        print(task_id)
        # print(task_id in child_task)

8bbbdd9d-e2e6-47a0-b33c-a21f306d7041
5c582020-20a9-464b-95fd-737fdae50993
9ae659af-39fe-4314-affc-44c44711b5ac
e1d404d6-cfcc-4ee2-9697-c575c7a98763
89da0e1b-0d4e-4f16-9f53-e1ab1f3df7c9
8571efe3-e07b-4d21-982c-c6a20f4167f1
a2c0a654-9596-4089-8f32-9162754c0ff0
2f3070e8-365f-4e79-999b-56be8827d8b2
dbba06bb-f114-4266-adba-e8b469b9bd98
0c8087e0-df44-444a-9c89-bcf76049775b
fd639cf2-dfb9-44f1-a3b0-c640ad89dd8e
37435edd-1b57-400a-b5da-86cd2a030620
d42c0898-6b3d-4a5e-aba6-867ab402f32c
fa0793cf-ae0a-4881-b374-9ed3bd019241
5ee44e70-93b2-4f84-b09e-55c7afeecfb8
ad20cea9-7d64-425a-94ae-04229c3e289e
8bbbdd9d-e2e6-47a0-b33c-a21f306d7041
5c582020-20a9-464b-95fd-737fdae50993
9ae659af-39fe-4314-affc-44c44711b5ac
e1d404d6-cfcc-4ee2-9697-c575c7a98763
89da0e1b-0d4e-4f16-9f53-e1ab1f3df7c9
8571efe3-e07b-4d21-982c-c6a20f4167f1
a2c0a654-9596-4089-8f32-9162754c0ff0
2f3070e8-365f-4e79-999b-56be8827d8b2
dbba06bb-f114-4266-adba-e8b469b9bd98
0c8087e0-df44-444a-9c89-bcf76049775b
fd639cf2-dfb9-44f1-a3b0-c640ad89dd8e
3

In [60]:
for _, message in records.items():
    print(f"==> Processing message: {type(message.decode('utf-8'))} and  {message} ")

==> Processing message: <class 'str'> and  b'{"task_id": 149559886724366975439206724654739506232, "task_name": "TaskSayHello", "group_task_id": null, "insert_dt": "2025-05-14T13:32:27.480557", "output_artifacts": "null"}' 
==> Processing message: <class 'str'> and  b'{"task_id": 35730254192003998894948829634200368416, "task_name": "TaskSayHello", "group_task_id": null, "insert_dt": "2025-05-14T13:38:51.553211", "output_artifacts": "null"}' 
==> Processing message: <class 'str'> and  b'{"task_id": 35730254192003998894948829634200368416, "task_name": "TaskSayHello", "group_task_id": null, "insert_dt": "2025-05-14T13:39:58.054279", "output_artifacts": "null"}' 
==> Processing message: <class 'str'> and  b'{"task_id": 300304694183836177550454581032852541384, "task_name": "DataColumnType", "group_task_id": null, "insert_dt": "2025-05-14T14:56:51.140983", "output_artifacts": "{\\"reason\\": \\"The \'membership\' column represents the kind of membership a gym member has, which would be a limi